<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 6.2: K-Means bad cases

INSTRUCTIONS:

- Replace the `___` with proper code to run the cells
- Observe and understand the results
- Answer the questions

In [0]:
## Import Libraries
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib.pyplot as plt

# from sklearn import datasets
from sklearn.datasets import make_circles
import seaborn as sns

In [0]:
## Create random data points (circles)
X1, _ = make_circles(n_samples = 400, factor = 0.3, noise = 0.05)

In [0]:
# Declare the columns names
columns = 'A B'.split()

# Load the dataset as a pandas data frame
X = pd.DataFrame(X1, columns = columns)

In [0]:
## Check data
print('X is a %s' % type(X))
print('X has %d rows and %d columns' % X.shape)
print('Basic Statistics about X%s' % ('_'*50))
print(X.describe())
print('')
print('Sample of X%s' % ('_'*50))
print(X.head())

In [0]:
sbn.pairplot(X)
plt.show()

### Question: What are other important things to know about the data? Comment on:
- Types
- Availability
- Dispersion and Distribution

### Question: Which features could/should be used to cluster the data points?

In [0]:
# Create new centroids randomly for all the K clusters including all dimensions
def create_centroids(K, features):
    
    centroids = [] # start with No centroids
    
    for i in range(K): # need one centroid for each cluster
        
        dimensions = [] # it can be multi-dimensional
        for f in features:
            # create a random number between the minimum and maximum of each feature
            x = np.random.uniform(X[f].min(), X[f].max())
            dimensions.append(x)
        
        centroids.append(dimensions) # add all dimensions for the centroids of each cluster

    return centroids

In [0]:
# Identify the closest centroid for a given data point
def closest_centroid(x, centroids):
    distances = []
    # convert x to a numpy.array; x contains all the features of a data point
    p = np.array(x)
    for c in centroids:
        # convert c to a numpy.array; c contains the coordinates for all the features
        q = np.array(c)
        # calculate the Euclidean distance between data point `x` and centroid `c`
        d = np.sqrt( ( (p - q) ** 2 ).sum() )
        # add the distance for each centroid
        distances.append(d)

    # return the position (cluster) which has the smallest distance
    return np.array(distances).argmin()

In [0]:
# recalculate all the centroids based on the mean of the members of each cluster
def update_centroid(X, y, centroids):
    c = []
    for i in range(len(centroids)): # for each of the existing centroids
        
        # check if any object was assigned to a cluster
        if X[y == i].shape[0]:
            dimensions = []
            for f in X.columns: # for all the features
                x = X[f][y == i].mean() # calculate the mean
                dimensions.append(x)
            c.append(dimensions)
        
        else: # if a cluster got no members
            #print('Keep', y, i)
            c.append(centroids[i]) # keep the previous coordinates

    return c

In [0]:
# plot a chart of the data points and centroids either generic or identifying the clusters
# NOTE: for visualisation purposes, only consider the first two features
def plot_data(ax, X, y, centroids, title, show_clusters=False):
        
    # unpack the coordinates of the centroids
    a = [z[0] for z in centroids]
    b = [z[1] for z in centroids]

    # show distinct colours if asked to identify the clusters
    if show_clusters:
        colours = 'rgbykcm'
        for i in range(len(centroids)):
            ax.scatter(X[features[0]][y == i], X[features[1]][y == i], c = colours[i], label = 'Cluster %d' % i)
    # show data points without cluster identification
    else:
        ax.scatter(X[features[0]], X[features[1]], label = 'Data Point')
        
    # label
    ax.set_xlabel(features[0])
    ax.set_ylabel(features[1])
    ax.set_title(title)

    # identify the centroids
    ax.scatter(a, b, c = 'k', linewidths = 5, marker = 'x', label = 'Centroid')
    for i in range(len(centroids)):
        c = colours[i] if show_clusters else 'k'
        ax.axvline(x = a[i], color = c, linestyle = '--', linewidth = 1)
        ax.axhline(y = b[i], color = c, linestyle = '--', linewidth = 1)
    
    # create a text to show the centroids' coordinates
    t = '\n'.join(['%d: (%.2f, %.2f)' % (i, a, b) for (i, (a, b)) in enumerate(centroids)])
    ax.text(0.69, -1.1, t)
    ax.legend(loc=2)

In [0]:
## K_Means
## Note that the comments match the outlined K-Means Algorithm in the presentation

# 1. Define the number of clusters `K`
K = ___ # choose different values for `K`

# list the features to be used
features = ['A', 'B']

# 2. Select `K` cluster centres randomly
centroids = create_centroids(K, features)
print("Initial centroids: \n{}".format('\n'.join(['(%.2f, %.2f)' % (c[0], c[1]) for c in centroids])))

# plot the data points and initial centroids (the 'Before')
fig, axes = plt.subplots(nrows = 1, ncols = 2, figsize = (18, 8))
plot_data(axes[0], X[features], None, centroids, title="Initial")

# repeat until makes no moves
while True:
    # 3. Calculate the distance between each data point and cluster centres
    # 4. Assign the data point to the cluster whose distance from its centre is minimum
    y = X[features].apply(lambda x: closest_centroid(x, centroids), axis = 1).values
    #print(y)
    
    # move the known assignment be the previous state, for post comparison
    old_c = centroids.copy()
    
    # 5. Recalculate all new cluster centres by averaging the objects of each cluster
    centroids = update_centroid(X[features], y, centroids)
    
    # 6. Repeat steps 3 to 5 until the centroids do not change
    if old_c == centroids:
        break
    
    # 7. For debugging
    if show_steps:
        print("Step {}".format(step))
        print('\n'.join(['(%.2f, %.2f)' % (c[0], c[1]) for c in centroids]))
    step += 1
    
# plot the data points identified by cluster and final centroids (the 'After')
plot_data(axes[1], X[features], y, centroids, title="Final", show_clusters=True)
plt.show()
print("Final centroids: \n{}".format('\n'.join(['(%.2f, %.2f)' % (c[0], c[1]) for c in centroids])))

### Question: What happens if the algorithm is run multiple times?
### Question: What happens if `K` is changed?

© 2019 Institute of Data